#### About maxvol_handmade
According to paper "How to find a good submatrix", written $maxvol\_handmade$ algorithm is built on following ideas:

* Consider we have matrix $A \in \mathbb{R}^{N \times r}$ and consider also that initial dominant sumbatrix $\hat A$ occupies first r rows of $A$. We need to compute matrix $C = A \hat A^{-1}$ with identity submatrix $I_{r\times r}$ in the top and submatrix $Z_{(N-r)\times r}$ in the bottom. According to $\textbf{Definition 3}$ from initial paper: We call $r \times r$ submatrix $\hat A$ of rectangular $N  \times r$ full rank matrix $A$ dominant, if all the entries of $C$ are not greater than 1 in modulus. And so if we manage to find $c_{ij}$ such that $|c_{ij}| > 1$, we can construct new submatrix with volume larger than volume of current top submatrix (for example, by swapping rows) and place it in the top of matrix $A$. 
* The most contributing part, if compute in naive way, is every time do $A \hat A^{−1}$ - inversion and multiplication. Sherman-Woodbury-Morrison formula for the matrix inverse can help us do nothing with stuff above but just with matrix $C$.

* Suppose we have found such $|c_{ij}| > 1$. We need now to swap $i$-th and $j$-th rows to get new submatrix, and we can proceed new $A \hat A^{−1}$ product just by:

$$
C = C - \frac{1}{c_{ij}} (c_{:, j} -e_j + e_i)(c_{i,:} - e_{j}^T),
$$

where $e_i$ and $e_j$ are columns with all zeros but ones in $i$ and $j$ positions respectively. And so $c_{:, j}$ and $c_{i,:}$ - $j$-th column and $i$-th row of $C$. 

Also on each step we need to put in memory number $i$ of row, that we swap with already present $j$-th row in top submatrix. At the output we will have indices of $r$ matrix $A$ rows , that will form dominant submatrix.

In [239]:
import numpy as np
import scipy

In [240]:
def maxvol_handmade(A, tol=1.05, max_iters=100, top_k_index=-1):

    # work on parameters
    if tol < 1:
        tol = 1.0
    N, r = A.shape
    if N <= r:
        return np.arange(N, dtype=np.int32), np.eye(N, dtype=A.dtype)
    if top_k_index == -1 or top_k_index > N:
        top_k_index = N
    if top_k_index < r:
        top_k_index = r
    
    #create matrix C with I and Z block
    B = np.copy(A[:top_k_index], order='F')
    B_inv = np.linalg.inv(B[:r])
    C = np.dot(B, B_inv)
    
    #initial indexing
    index = np.arange(N, dtype=np.int32)
      
    # find initial max value in C, if it exists
    i, j = divmod(abs(C[:,:]).argmax(), r)
        
    # set number of iters to 0
    iters = 0
      
    # check if need to swap rows
    while abs(C[i,j]) > tol and iters < max_iters:
        # add i to index and recompute C by SWM-formula
        index[j] = i
        
        tmp_row = C[i, :].copy()
        tmp_row[j] -= 1.
        
        tmp_column = C[:,j].copy()
        tmp_column[j] -= 1.
        tmp_column[i] += 1.
        
        tmp_column = np.matrix(tmp_column).T
        tmp_row = np.matrix(tmp_row)
        
        alpha = -1./C[i,j]
        C += alpha*tmp_column.dot(tmp_row)
        
        iters += 1
        i, j = divmod(abs(C[:,:]).argmax(), r)
    return index[:r]
